In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Load the dataset
dataset_path = Path('future_skills_dataset.csv')
if not dataset_path.exists():
    print("⚠️ Dataset not found. Please run: python ../manage.py export_future_skills_dataset")
else:
    df = pd.read_csv(dataset_path)
    print(f"✅ Dataset loaded: {len(df)} rows, {len(df.columns)} columns")
    print(f"\nColumns: {', '.join(df.columns)}")

## 1️⃣ Basic Dataset Information

In [ ]:
# Display basic info
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Data types and missing values
print("Data Types and Missing Values:")
info_df = pd.DataFrame({
    'Type': df.dtypes,
    'Missing': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df) * 100).round(2)
})
info_df

In [ ]:
# Statistical summary of numeric features
print("\nStatistical Summary:")
df.describe()

## 2️⃣ Target Variable Analysis - Class Distribution

In [ ]:
# Class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
class_counts = df['future_need_level'].value_counts()
sns.barplot(x=class_counts.index, y=class_counts.values, ax=axes[0], palette='viridis')
axes[0].set_title('Distribution of Future Need Level', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Need Level')
axes[0].set_ylabel('Count')

# Add value labels on bars
for i, v in enumerate(class_counts.values):
    axes[0].text(i, v + 0.5, str(v), ha='center', fontweight='bold')

# Pie chart
colors = ['#ff6b6b', '#ffd93d', '#6bcf7f']
axes[1].pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90)
axes[1].set_title('Percentage Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Print statistics
print("\nClass Distribution:")
for level, count in class_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {level}: {count} ({percentage:.1f}%)")

# Check for imbalance
max_class = class_counts.max()
min_class = class_counts.min()
imbalance_ratio = max_class / min_class
print(f"\n⚖️ Imbalance Ratio: {imbalance_ratio:.2f}")
if imbalance_ratio > 3:
    print("⚠️ WARNING: Significant class imbalance detected!")
else:
    print("✅ Classes are reasonably balanced.")

## 3️⃣ Categorical Features Analysis

In [ ]:
# Analyze job roles
print("Top 10 Job Roles by Frequency:")
job_role_counts = df['job_role_name'].value_counts().head(10)
print(job_role_counts)

plt.figure(figsize=(12, 6))
sns.barplot(x=job_role_counts.values, y=job_role_counts.index, palette='coolwarm')
plt.title('Top 10 Job Roles', fontsize=14, fontweight='bold')
plt.xlabel('Count')
plt.ylabel('Job Role')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze skills
print("\nTop 10 Skills by Frequency:")
skill_counts = df['skill_name'].value_counts().head(10)
print(skill_counts)

plt.figure(figsize=(12, 6))
sns.barplot(x=skill_counts.values, y=skill_counts.index, palette='plasma')
plt.title('Top 10 Skills', fontsize=14, fontweight='bold')
plt.xlabel('Count')
plt.ylabel('Skill')
plt.tight_layout()
plt.show()

In [ ]:
# Skill categories and job departments
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Skill categories
if 'skill_category' in df.columns:
    skill_cat_counts = df['skill_category'].value_counts()
    sns.barplot(x=skill_cat_counts.values, y=skill_cat_counts.index, ax=axes[0], palette='viridis')
    axes[0].set_title('Skill Categories', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Count')

# Job departments
if 'job_department' in df.columns:
    dept_counts = df['job_department'].value_counts()
    sns.barplot(x=dept_counts.values, y=dept_counts.index, ax=axes[1], palette='magma')
    axes[1].set_title('Job Departments', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Count')

plt.tight_layout()
plt.show()

## 4️⃣ Numeric Features Analysis

In [ ]:
# Select numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"Numeric features: {', '.join(numeric_cols)}")

# Distribution of numeric features
n_cols = len(numeric_cols)
n_rows = (n_cols + 2) // 3
fig, axes = plt.subplots(n_rows, 3, figsize=(15, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    axes[i].hist(df[col], bins=30, edgecolor='black', color='steelblue', alpha=0.7)
    axes[i].set_title(f'Distribution of {col}', fontweight='bold')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')
    
    # Add statistics
    mean_val = df[col].mean()
    median_val = df[col].median()
    axes[i].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
    axes[i].axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Median: {median_val:.2f}')
    axes[i].legend()

# Hide unused subplots
for i in range(n_cols, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 5️⃣ Outlier Detection

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(n_rows, 3, figsize=(15, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(numeric_cols):
    sns.boxplot(y=df[col], ax=axes[i], color='lightblue')
    axes[i].set_title(f'Box Plot - {col}', fontweight='bold')
    axes[i].set_ylabel(col)
    
    # Calculate and display outliers
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)]
    axes[i].text(0.05, 0.95, f'Outliers: {len(outliers)}', 
                transform=axes[i].transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Hide unused subplots
for i in range(n_cols, len(axes)):
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Detailed outlier statistics
print("\nOutlier Statistics (using IQR method):")
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_percentage = (len(outliers) / len(df)) * 100
    print(f"  {col}: {len(outliers)} outliers ({outlier_percentage:.2f}%)")

## 6️⃣ Feature Correlations

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 10))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Print strong correlations
print("\nStrong Correlations (|correlation| > 0.5):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.5:
            print(f"  {correlation_matrix.columns[i]} ↔ {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

## 7️⃣ Feature Distribution by Target Class

In [ ]:
# Box plots by target class
n_cols_show = min(6, len(numeric_cols))  # Show first 6 numeric features
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, col in enumerate(numeric_cols[:n_cols_show]):
    sns.boxplot(x='future_need_level', y=col, data=df, ax=axes[i], 
                palette='Set2', order=['LOW', 'MEDIUM', 'HIGH'])
    axes[i].set_title(f'{col} by Need Level', fontweight='bold')
    axes[i].set_xlabel('Future Need Level')

plt.tight_layout()
plt.show()

## 8️⃣ Data Quality Checks

In [ ]:
print("\n📋 DATA QUALITY REPORT\n" + "="*50)

# 1. Missing values
missing_count = df.isnull().sum().sum()
print(f"\n1. Missing Values: {missing_count}")
if missing_count == 0:
    print("   ✅ No missing values detected")
else:
    print("   ⚠️ Missing values found:")
    print(df.isnull().sum()[df.isnull().sum() > 0])

# 2. Duplicate rows
duplicates = df.duplicated().sum()
print(f"\n2. Duplicate Rows: {duplicates}")
if duplicates == 0:
    print("   ✅ No duplicate rows")
else:
    print("   ⚠️ Duplicate rows found")

# 3. Target class distribution
print(f"\n3. Target Class Balance:")
class_dist = df['future_need_level'].value_counts(normalize=True) * 100
for level, pct in class_dist.items():
    status = "✅" if 20 <= pct <= 50 else "⚠️"
    print(f"   {status} {level}: {pct:.1f}%")

# 4. Feature value ranges
print(f"\n4. Feature Value Ranges:")
for col in numeric_cols:
    min_val = df[col].min()
    max_val = df[col].max()
    # Check if values are in expected range [0, 1] for most features
    if col != 'avg_salary_k':
        status = "✅" if 0 <= min_val and max_val <= 1 else "⚠️"
        print(f"   {status} {col}: [{min_val:.3f}, {max_val:.3f}]")
    else:
        status = "✅" if min_val > 0 and max_val < 200 else "⚠️"
        print(f"   {status} {col}: [{min_val:.2f}, {max_val:.2f}] K€")

# 5. Unique combinations
unique_combos = df[['job_role_name', 'skill_name']].drop_duplicates().shape[0]
print(f"\n5. Unique (Job Role, Skill) Combinations: {unique_combos}")
print(f"   Total rows: {len(df)}")
if unique_combos == len(df):
    print("   ✅ All combinations are unique")
else:
    print("   ⚠️ Some combinations appear multiple times")

print("\n" + "="*50)

## 9️⃣ Summary and Recommendations

In [ ]:
print("\n🎯 DATASET SUMMARY AND RECOMMENDATIONS\n" + "="*60)

print(f"\n📊 Dataset Size:")
print(f"   • Total samples: {len(df)}")
print(f"   • Features: {len(df.columns)}")
print(f"   • Unique job roles: {df['job_role_name'].nunique()}")
print(f"   • Unique skills: {df['skill_name'].nunique()}")

print(f"\n🎯 Target Distribution:")
class_counts = df['future_need_level'].value_counts()
for level in ['LOW', 'MEDIUM', 'HIGH']:
    if level in class_counts:
        count = class_counts[level]
        pct = (count / len(df)) * 100
        print(f"   • {level}: {count} ({pct:.1f}%)")

print(f"\n💡 Recommendations:")

# Check class imbalance
imbalance_ratio = class_counts.max() / class_counts.min()
if imbalance_ratio > 3:
    print("   ⚠️ CLASS IMBALANCE DETECTED (ratio > 3)")
    print("      → Consider using class_weight='balanced' in the model")
    print("      → Consider SMOTE or other resampling techniques")
    print("      → Use stratified train/test split")
else:
    print("   ✅ Classes are reasonably balanced")

# Check dataset size
if len(df) < 100:
    print("   ⚠️ SMALL DATASET (< 100 samples)")
    print("      → Add more job roles and skills to increase diversity")
    print("      → Consider using cross-validation instead of train/test split")
elif len(df) < 500:
    print("   ⚠️ MODERATE DATASET SIZE (< 500 samples)")
    print("      → Use cross-validation for better model evaluation")
    print("      → Consider data augmentation if possible")
else:
    print("   ✅ Good dataset size for training")

# Check for outliers
total_outliers = 0
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = len(df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)])
    total_outliers += outliers

if total_outliers > len(df) * 0.05:  # More than 5% outliers
    print("   ⚠️ MANY OUTLIERS DETECTED (> 5% of data)")
    print("      → Review outliers to ensure they're realistic")
    print("      → Consider robust scaling or outlier removal")
else:
    print("   ✅ Acceptable number of outliers")

print("\n✨ Next Steps:")
print("   1. If satisfied with the data quality, proceed with model training")
print("   2. Update the training script to use new features")
print("   3. Run: python ../manage.py export_future_skills_dataset")
print("   4. Run: python train_future_skills_model.py")
print("   5. Evaluate model performance and iterate")

print("\n" + "="*60)